In [1]:
import os
import sys
import json
import stanza
from tqdm import tqdm

w_dir = %pwd
work_dir = os.path.dirname(w_dir)
work_dir

'/media/moju/data/work/ser'

In [2]:
sys.path.append(w_dir+'/ser_annotate')

In [3]:
from ser_annotate.dataset_reader.ssqa_old_bench import read_old_benchmark
from ser_annotate.dataset_reader.ssqa_xml import ssqa_xml2txt
from ser_annotate import config
from ser_annotate.dataset_reader.dataset import CanoDict

2020-04-28 17:44:05 INFO: "zh" is an alias for "zh-hans"
2020-04-28 17:44:05 INFO: Loading these models for language: zh-hans (Simplified_Chinese):
| Processor | Package   |
-------------------------
| tokenize  | gsdsimp   |
| ner       | ontonotes |

2020-04-28 17:44:05 INFO: Use device: cpu
2020-04-28 17:44:05 INFO: Loading: tokenize
2020-04-28 17:44:05 INFO: Loading: ner
2020-04-28 17:44:05 INFO: Done loading processors!


In [27]:
benchs = read_old_benchmark(config.SSQA_OLD_BENCH_TEST)

ssqa_old_bench.py(115): #Lesson:27
ssqa_old_bench.py(116): #Question:207


In [28]:
 benchs['PubB-G6a-0302'][1]

KeyError: 'PubB-G6a-0302'

In [29]:
def inspect_old_bench(benches):
    num_p = 0
    num_s = 0
    num_q = 0
    num_l = len(benches)

    for lesson_id, lesson in benches.items():
        parags = lesson[0]
        Qs = lesson[1]
        
        num_q += len(Qs)
        num_p += len(parags)

        for pid, sents in parags:
            num_s += len(sents)

    print("#Lesson:{}".format(num_l))
    print("#Question:{}".format(num_q))
    print("Mean #paragraphs per lesson:{}".format(num_p/num_l))
    print("Mean #sentences per paragraph:{}".format(num_s/num_p))

In [30]:
inspect_old_bench(benchs)

#Lesson:27
#Question:207
Mean #paragraphs per lesson:28.77777777777778
Mean #sentences per paragraph:3.6911196911196913


In [8]:
# for qid, q_text, se in benchs['PubB-G6a-0302'][1]:
#     print(qid)
#     print(q_text)
#     for parag in se:
#         pid, p_text, s_ind = parag
#         print(pid)
#         print(p_text)
#         print(s_ind)
#     print('========')

In [9]:
test_file = './data/SSQA/Elementary_Social_Studies_v2.9/Develop/PubB-G6a-0302.xml'

In [10]:
parags, lessonTxt, q_dict = ssqa_xml2txt(test_file)

In [11]:
q_dict.keys()

dict_keys(['true-false', 'multiple-choice', 'multiple-select'])

In [12]:
# lessonTxt

In [13]:
# parags

In [14]:
q_dict['multiple-select']

{'06-002485': ('有鑒於社會上單親家庭愈來愈多，政府、社會和學校應該從哪些方面幫助單親家庭？',
  ['學校和社會教育機構積極規畫並實施單親家長的親職教育方案',
   '運用大眾媒體的力量，廣泛的提供單親家庭有關親職教育的資訊',
   '不與單親家庭成員打交道',
   '藉由社區力量和資源，建立協助單親家長的支持組織',
   '政府制定福利方案，整合社會福利、教育、醫療、法律等服務體系，提供完整服務',
   '政府提供生活費給單親家庭，並限制單親家長只能在家照顧小孩，不可外出工作'],
  [0, 1, 3, 4]),
 '06-002486': ('造成近年來隔代教養家庭增多的原因有哪些？',
  ['父母親到都市就業，將小孩託付給祖父母照顧',
   '高齡化社會來臨，政府鼓勵祖父母協助照顧孫子女',
   '父母親離婚，將小孩託付給祖父母照顧',
   '每戶家庭平均人口數變少'],
  [0, 2]),
 '06-002487': ('促使女性就業人口增加的原因有哪些？',
  ['女性教育程度提高', '性別平等觀念的提倡', '「男主內、女主外」觀念的提倡', '男性人口大幅減少', '人口平均壽命延長'],
  [0, 1]),
 '06-002488': ('婦女在職場遇到哪些問題時，可以依據性別工作平等法來爭取自己的權益？',
  ['玉珍懷孕要生產了，老闆讓她放產假',
   '水電行拒絕聘請女性職員當員工',
   '醫學院畢業的女性在醫院擔任醫生',
   '某五星級飯店不讓女性當主廚',
   '客運公司拒絕任用女性當司機'],
  [1, 3, 4])}

In [15]:
set_list = ['Train', 'Develop', 'Test']

In [16]:
def write_jsonl(did, ssqa_xml, out_fp):
    parags, lessonTxt, q_dict = ssqa_xml2txt(ssqa_xml)
    
    out_canos = []
    with open(out_fp, 'w') as f:
        for qid, qa_pair in q_dict['true-false'].items():
            cano = CanoDict()
            cano.qid = did + '-' + qid
            cano.c_text = lessonTxt
            cano.q_text = qa_pair[0]
            cano.a_list.append(qa_pair[1])
            cano.merge_q()
            out_canos.append(cano)
        out_canos = out_canos[:1]
        
        for qid, qa_pair in q_dict['multiple-choice'].items():
            cano = CanoDict()
            cano.qid = did + '-' + qid
            cano.c_text = lessonTxt
            cano.q_text = qa_pair[0]
            cano.choices = qa_pair[1]
            cano.a_list = [qa_pair[1][qa_pair[2]]]
            cano.merge_q()
            out_canos.append(cano)
        out_canos = out_canos[:2]    
        
        for qid, qa_pair in q_dict['multiple-select'].items():
            cano = CanoDict()
            cano.qid = did + '-' + qid
            cano.c_text = lessonTxt
            cano.q_text = qa_pair[0]
            cano.choices = qa_pair[1]
            cano.a_list = [qa_pair[1][i] for i in qa_pair[2]]
            cano.merge_q()
            out_canos.append(cano)
        out_canos = out_canos[:3]    
    
        for cano in out_canos:
            json.dump(cano.cano_format, f, ensure_ascii=False)
            f.write('\n')

In [17]:
write_jsonl('PubB-G6a-0302', test_file, 'PubB-G6a-0302.jsonl')

In [17]:
q_num = 0

for i_set in set_list:
    type_counter = Counter()
    for fp in tqdm(os.listdir(ssqa_fp+i_set)):
        fp_c = ssqa_fp+i_set + '/' + fp
        try:
            p = ET.parse(fp_c).getroot()
            assert p[0][0][1].tag == 'QAset'
            q_num += len(p[0][0][1])
            lesson_xml2txt(fp_c, type_counter)
        except Exception as e:
            print(e)
            print(fp_c)
    print(i_set)
    print(type_counter)

 57%|█████▋    | 119/210 [00:01<00:01, 81.18it/s]<ipython-input-12-3c246c7e2016>(26): PubC-G3b-0602
	對話框:"各組想想看，有什麼解決的方法？">>"我們可以寫信給區長，請他幫忙。">>"製作海報張貼在社區。">>"請學校發起淨灘活動。"
	對話框:"各組想想看，有什麼解決的方法？"&gt;&gt;"我們可以寫信給區長，請他幫忙。"&gt;&gt;"製作海報張貼在社區。"&gt;&gt;"請學校發起淨灘活動。"

 33%|███▎      | 9/27 [00:00<00:00, 86.78it/s]

Train
Counter({'true-false': 11131, 'multiple-choice': 4022, 'essay': 999, 'multiple-select': 191})


 37%|███▋      | 10/27 [00:00<00:00, 97.40it/s]

Develop
Counter({'true-false': 1508, 'multiple-choice': 789, 'essay': 208, 'multiple-select': 81})


100%|██████████| 27/27 [00:00<00:00, 78.91it/s]

Test
Counter({'true-false': 1464, 'multiple-choice': 788, 'essay': 254, 'multiple-select': 68})


In [18]:
q_num

21503